<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Neighborhoods data is processed to Pandas dataframe from notebook :
https://github.com/EdwinLife/Coursera_Capstone/blob/master/Explore%20Neighborhoods%20in%20Toronto%20-%20Web%20Scraping.ipynb

Here latitude and longitude of Toronto's neighborhoods is obtained from csv file referred in assignment. 

#### Code in first Notebook scraping wikipedia table to Pandas dataframe.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#scrape table from wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).content, 'lxml')
table = soup.find_all('table')[0] 
df = pd.DataFrame()

header = []
for th in table.find_all("th"):
    # remove new line and spaces from left and right        
    header.append(th.text.replace('\n', ' ').strip())

for tr in table.tbody.find_all("tr"): # find all <tr> from table's tbody
    t_row = {}
    add_row = True
    for td, th in zip(tr.find_all("td"), header): 
        t_row[th] = td.text.replace('\n', '').strip()
        if th == 'Borough' and (t_row[th] == 'Not assigned' or t_row[th] == ''):
            add_row = False
        
    if add_row:
        df = df.append(t_row, ignore_index=True)

df = df[header]
df = df.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')
df.rename(columns={'Postal code':'Postal Code'}, inplace='True')
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df.shape

(103, 3)

Geocoder Python package instead: https://geocoder.readthedocs.io/index.html seems to be unreliable as mentioned in Assignment instruction, hence the csv file mentioned is used - that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [5]:
import numpy as np
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

df_join = pd.merge(df, df_geo, on='Postal Code', how='left')
df_join.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848
